In [ ]:
import pandas as pd
import re
import csv

### Outlier decetion of internal data

In [ ]:
# Converte json document to csv.
pdObj = pd.read_json('../data/curated/domain.json', orient='index')
pdObj.to_csv('../data/curated/streaming_test.csv', index=False)

In [ ]:
# Get the cost
df = pd.read_csv('../data/curated/streaming_test.csv')
df['cost'] = df['cost_text']
for i in range(len(df)):
    df['cost'].iloc[i] = re.findall(r'\d+\.?\d*',df['cost_text'].iloc[i])

In [ ]:
# Delete missing values if cost is 0
delete_list = []
df_new = df.copy()
for i in range(len(df)):
    if len(df['cost'].iloc[i]) == 0:
        delete_list.append(i)
df_new = df_new.drop(df_new.index[delete_list])

In [ ]:
# Transform the data type of cost into float.
for i in range(len(df_new)):
    df_new['cost'].iloc[i] = float(df_new['cost'].iloc[i][0])
    
df_new['cost'] = df_new['cost'].astype('float')

In [ ]:
# Delete the extreme values: delete Highest 20% and lowest 20% cost.
delete_list = []
k1 = 0.8
k2 = 0.2
df_new2 = df_new.copy()
for i in range(len(df_new2)):
    if df_new2['cost'].iloc[i] < df_new2['cost'].mean()*(1-k1) or df_new2['cost'].iloc[i] > df_new2['cost'].mean()*(1+k2):
        delete_list.append(i)
df_new2 = df_new2.drop(df_new2.index[delete_list])
df_new2.describe()

In [ ]:
# Divide rooms into 'Bed', 'Bath' and 'Parking'
df_new2['Beds'] = df_new2['rooms']
df_new2['Bath'] = df_new2['rooms']
df_new2['Parking'] = df_new2['rooms']

for i in range(len(df_new2)):
    try:
        df_new2['Beds'].iloc[i] = eval(df_new2['rooms'].iloc[i])[0][0]
    except:
        df_new2['Beds'].iloc[i] = 0
    try:
        df_new2['Bath'].iloc[i] = eval(df_new2['rooms'].iloc[i])[1][0]
    except:
        df_new2['Bath'].iloc[i] = 0
    try:
        df_new2['Parking'].iloc[i] = eval(df_new2['rooms'].iloc[i])[2][0]
    except:
        df_new2['Parking'].iloc[i] = 0

In [ ]:
domain = df_new2

In [ ]:
domain.head()

Some suburb data is missing in domian, this is probably because there are not enough property transactions in this postcide suburb on domian to be counted, so we try to fill in the data with neighbouring suburb, or delete them if there are no near suburb or only a few data is missing

In [ ]:
domain[domain['suburb_population'].isnull()]['postcode'].value_counts()

In [ ]:
# 3000 data is filled in 3004
domain.loc[domain['postcode'] == 3004, 'suburb_sold'] = 596
domain.loc[domain['postcode'] == 3004, 'suburb_avg_day'] = 147
domain.loc[domain['postcode'] == 3004, 'suburb_population'] = 6423
domain.loc[domain['postcode'] == 3004, 'suburb_age'] = '20 to 39'
domain.loc[domain['postcode'] == 3004, 'suburb_owner'] = '30%'
domain.loc[domain['postcode'] == 3004, 'suburb_renter'] = '70%'
domain.loc[domain['postcode'] == 3004, 'suburb_family'] = '24%'
domain.loc[domain['postcode'] == 3004, 'suburb_single'] = '76%'

In [ ]:
domain.loc[domain['postcode'] == 3046, 'suburb_sold'] = 178
domain.loc[domain['postcode'] == 3046, 'suburb_avg_day'] = 48
domain.loc[domain['postcode'] == 3046, 'suburb_population'] = 22246
domain.loc[domain['postcode'] == 3046, 'suburb_age'] = '20 to 39'
domain.loc[domain['postcode'] == 3046, 'suburb_owner'] = '63%'
domain.loc[domain['postcode'] == 3046, 'suburb_renter'] = '37%'
domain.loc[domain['postcode'] == 3046, 'suburb_family'] = '49%'
domain.loc[domain['postcode'] == 3046, 'suburb_single'] = '51%'

In [ ]:
domain.loc[domain['postcode'] == 3024, 'suburb_sold'] = 288
domain.loc[domain['postcode'] == 3024, 'suburb_avg_day'] = 40
domain.loc[domain['postcode'] == 3024, 'suburb_population'] = 23294
domain.loc[domain['postcode'] == 3024, 'suburb_age'] = '20 to 39'
domain.loc[domain['postcode'] == 3024, 'suburb_ower'] = '72%'
domain.loc[domain['postcode'] == 3024, 'suburb_renter'] = '28%'
domain.loc[domain['postcode'] == 3024, 'suburb_family'] = '51%'
domain.loc[domain['postcode'] == 3024, 'suburb_single'] = '49%'

In [ ]:
domain.loc[domain['postcode'] == 3337, 'suburb_sold'] = 99
domain.loc[domain['postcode'] == 3337, 'suburb_avg_day'] = 43
domain.loc[domain['postcode'] == 3337, 'suburb_population'] = 17589
domain.loc[domain['postcode'] == 3337, 'suburb_age'] = '20 to 39'
domain.loc[domain['postcode'] == 3337, 'suburb_ower'] = '74%'
domain.loc[domain['postcode'] == 3337, 'suburb_renter'] = '26%'
domain.loc[domain['postcode'] == 3337, 'suburb_family'] = '46%'
domain.loc[domain['postcode'] == 3337, 'suburb_single'] = '54%'

In [ ]:
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_sold'].isnull()), 'suburb_sold'] = 97
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_avg_day'].isnull()),'suburb_avg_day'] = 14
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_population'].isnull()),'suburb_population'] = 8343
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_age'].isnull()),'suburb_age'] = '60+'
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_ower'].isnull()), 'suburb_ower'] = '80%'
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_renter'].isnull()),'suburb_renter'] = '20%'
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_family'].isnull()),'suburb_family'] = '51%'
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_single'].isnull()),'suburb_single'] = '49%'

In [ ]:
for post in domain.postcode:
    domain =  domain.drop(domain[(domain['postcode'] == post)&(domain['suburb_population'].isnull())].index)

In [ ]:
domain.to_csv('../data/curated/domain_final.csv',index=False)

### Locality to SA2 Coding Index
a csv saving convertation between SA2 and postcode

In [ ]:
sa2_2016 = pd.read_csv("../data/raw/australian_postcodes.csv") # SA2 code in 2016
sa2_2021 = pd.read_csv("../data/raw/CG_SA2_2016_SA2_2021.csv")# SA2 code in 2021

In [ ]:
# merge data and only keep SA2 in VIC
index = sa2_2021.merge(sa2_2016, on='SA2_MAINCODE_2016')
index = index[index['state'] == 'VIC']

In [ ]:
# keep useful col and drop duplicates
index = index[['SA2_MAINCODE_2016', 'SA2_NAME_2016_x', 'SA2_CODE_2021', 'SA2_NAME_2021','postcode','locality','Lat_precise','Long_precise']]
index = index.drop_duplicates(subset=['postcode'])

In [ ]:
postcode = pd.read_csv('../data/raw/Australian-Postcode-Data-master/au_postcodes.csv') # postcode in VIC
code_vic = postcode[postcode['state_code'] == 'VIC']
code_vic = code_vic[postcode['accuracy'] == 4] # only keep accuracy is 4 postcode that refer to a suburb
code_vic = code_vic.drop_duplicates(subset=['postcode'])

In [ ]:
index_vic = code_vic[['postcode','latitude','longitude']].merge(index2, on='postcode')

In [ ]:
index_vic
index_vic.to_csv('../data/curated/postcode_to_sa2.csv',index=False)